In [2]:
import openpyxl as opxl
import pandas as pd
import re
import numpy as np
import unicodedata
from IPython.display import display, HTML

In [11]:
def convert_to_lower_case(data):
    if type(data) is dict:
        for k, v in data.items():
            if type(v) is str:
                data[k] = v.lower()
            elif type(v) is list:
                data[k] = [x.lower() for x in v]
            elif type(v) is dict:
                data[k] = convert_to_lower_case(v)
    return data

In [30]:
files_to_read = [
    {
        'output_file' : '2014-15_gems_jade', \
        'year': '2014-2015', \
        'xls_file' : 'Annex 18 - Reconciliation sheets Gems & Jade 14-15.xlsx', \
        'name_cols': 'B:D', \
        'data_cols' : 'A:M', \
        'data_skip_rows' : 4, \
        'stop_at' : '1000'
    },
    {
        'output_file' : '2014-15_oil_gas', \
        'year': '2014-2015', \
        'xls_file' : 'Annex 18 - Reconciliation sheets Oil & Gas 14-15.xlsx', \
        'name_cols': 'B:E', \
        'data_cols' : 'A:M', \
        'data_skip_rows' : 5, \
        'stop_at' : '1000'
    },
    {
        'output_file' : '2014-15_other_minerals', \
        'year': '2014-2015', \
        'xls_file' : 'Annex 18 - Reconciliation sheets Other minerals 14-15.xlsx', \
        'name_cols': 'B:E', \
        'data_cols' : 'A:M', \
        'data_skip_rows' : 5, \
        'stop_at' : '1000'
    },
    {
        'output_file' : '2015-16_gems_jade', \
        'year': '2015-2016', \
        'xls_file' : 'Annex 18 - Reconciliation sheets Gems & Jade 15-16.xlsx', \
        'name_cols': 'C:E', \
        'data_cols' : 'A:N', \
        'data_skip_rows' : 4, \
        'stop_at' : '51'
    },
    {
        'output_file' : '2015-16_oil_gas', \
        'year': '2015-2016', \
        'xls_file' : 'Annex 18 - Reconciliation sheets Oil & Gas 15-16.xlsx', \
        'name_cols': 'B:E', \
        'data_cols' : 'A:M', \
        'data_skip_rows' : 5, \
        'stop_at' : '35'
    },
    {
        'output_file' : '2015-16_oil_gas_transport', \
        'year': '2015-2016', \
        'xls_file' : 'Annex 18 - Reconciliation sheets Oil & Gas transp 15-16.xlsx', \
        'name_cols': 'B:E', \
        'data_cols' : 'A:M', \
        'data_skip_rows' : 5, \
        'stop_at' : '5'
    },
    {
        'output_file' : '2015-16_other_minerals', \
        'year': '2015-2016', \
        'xls_file' : 'Annex 18 - Reconciliation sheets Other minerals 15-16.xlsx', \
        'name_cols': 'B:E', \
        'data_cols' : 'A:M', \
        'data_skip_rows' : 5, \
        'stop_at' : '28'
    }
]

'''
files_to_read = [
    {
        'output_file' : '2015-16_other_minerals', \
        'year': '2015-2016', \
        'xls_file' : 'Annex 18 - Reconciliation sheets Other minerals 15-16.xlsx', \
        'name_cols': 'B:E', \
        'data_cols' : 'A:M', \
        'data_skip_rows' : 5, \
        'stop_at' : '28'
    }
]


    {
        'output_file' : '2014-15_oil_gas_transport', \
        'year': '2014-2015', \
        'xls_file' : 'Annex 18 - Reconciliation sheets Oil & Gas transp 15-16.xlsx', \
        'name_cols': 'B:E', \
        'data_cols' : 'A:M', \
        'data_skip_rows' : 5, \
        'stop_at' : '1000'
    },

'''

#name_cols = 'B:D'
#data_cols = 'A:M'
#data_skip_rows = 4

"\nfiles_to_read = [\n    {\n        'output_file' : '2015-16_other_minerals',         'year': '2015-2016',         'xls_file' : 'Annex 18 - Reconciliation sheets Other minerals 15-16.xlsx',         'name_cols': 'B:E',         'data_cols' : 'A:M',         'data_skip_rows' : 5,         'stop_at' : '28'\n    }\n]\n\n\n    {\n        'output_file' : '2014-15_oil_gas_transport',         'year': '2014-2015',         'xls_file' : 'Annex 18 - Reconciliation sheets Oil & Gas transp 15-16.xlsx',         'name_cols': 'B:E',         'data_cols' : 'A:M',         'data_skip_rows' : 5,         'stop_at' : '1000'\n    },\n\n"

In [31]:
df_all = pd.DataFrame({'company_name' : [],'name_of_revenue_stream' : [], \
                       'paid_to' : [], 'payment_category' : [], 'units' : [], \
                       'per_company_original' : [], \
                       'per_company_adjust' : [], \
                       'per_company_final' : [], \
                       'per_government_original' : [], \
                       'per_government_adjust' : [], \
                       'per_government_final' : [], \
                       'final_difference' : [], \
                       'comment' : []
                      })

key_terms = { 'payment_category' : ['Payments in kind', 'Payments in cash', 'B- Unilateral company disclosures'], \
              'units' : ['In Barils', 'In Mscf', 'Gold in T.oz', 'Tin in MT', 'In (Please mention the commodity)', \
                       'Antimony Ore', 'NA', 'Copper', 'Copper in MT', 'Ferro Nickel']
            }
key_terms = convert_to_lower_case(key_terms)

In [32]:
def join_column_titles(text):
    #print(text)
    to_return = ""
    for t in text:
        #print(t)
        if not isinstance(t, float):
            to_return = '.'.join([to_return,(''.join(i for i in t if ord(i)<128))])
    #print("FINAL: " + to_return)
    
    # remove first '.' from the title
    return to_return[1:]

In [33]:
def rename_duplicate_column_titles(columns):
    unique_titles = []
    title_counts = {}
    
    for c in columns:
        if c in unique_titles:
            title_counts[c] += 1
            unique_titles.append(c + "." + str(title_counts[c]))
        else:
            title_counts[c] = 0
            unique_titles.append(c)
            
    return unique_titles

In [34]:
def add_sheet_to_main_df(main_df,current_df, company_name,key_terms):

    
    current_payment_category = ""
    current_units = "MMK"
    current_paid_to = ""

            
    for index, row in current_df.iterrows():
        
        #display(row)
            
        index_col = 'n'
        description_col = 'description of payment'
        
        # skip row if '°' is included in it because that means 
        #  there's an extra row of French titles in the table
        unicode_row = row.to_string().encode("utf-8")        
        if u'\xb0' in unicode_row.decode('windows-1252'):
            continue
        
        # if the index column in empty, that means it's not a data-row
        if str(row[index_col])  == 'nan':
            index_col = description_col
        
        #print(str(row[description_col]).lower() + " | " + str(row[index_col]).lower())
        
        if str(row[index_col]).lower() in key_terms['payment_category']:
            current_payment_category = str(row[index_col]).lower()
        elif str(row[index_col]).lower() in key_terms['units']:
            current_units = str(row[index_col]).lower()
            current_paid_to = ""
        elif not str(row[index_col]).replace('.','',1).isdigit():
            current_paid_to = str(row[index_col])
            current_units = "MMK"
        
        if str(row['n']).replace('.','',1).isdigit():
            to_append = pd.DataFrame({'company_name' : [company_name], \
                                      'name_of_revenue_stream' : [row['description of payment']] , \
                                      'paid_to' : [current_paid_to], \
                                      'payment_category' : [current_payment_category], \
                                      'units' : [current_units], \
                                      'per_company_original' : [row['per company.original']], \
                                      'per_company_adjust' : [row['per company.adjust']], \
                                      'per_company_final' : [row['per company.final']], \
                                      'per_government_original' : [row['per government.original']], \
                                      'per_government_adjust' : [row['per government.adjust']], \
                                      'per_government_final' : [row['per government.final']], \
                                      'final_difference' : [row['final difference']], \
                                      'comment' : [row['comment']] \
                                    })
            
            
            to_append['comment'].fillna('', inplace=True)
            to_append.fillna(0, inplace=True)
            
            main_df = pd.concat([main_df, to_append])
        
    return main_df
    
    

In [47]:
def read_files(files_to_read, df_all): 
    for f in files_to_read:

        output_file = f['output_file']
        year = f['year']
        xls_file = f['xls_file']
        name_cols = f['name_cols']
        data_cols = f['data_cols']
        data_skip_rows = f['data_skip_rows']   
        stop_at = f['stop_at']       

        xl = pd.ExcelFile(year+'/'+xls_file)
#        '2014-2015/Annex 18 - Reconciliation sheets Gems & Jade 14-15.xlsx')

        print(year+'/'+xls_file)
        sheets_list = xl.sheet_names
    
        print(sheets_list)

        df_all = pd.DataFrame()
        for s in sheets_list:
            
            if not re.search("[A-Z]+\s\(\d+\)",s):
                continue
            else:
                match= re.search(r'(\d+)', s)
                print('current sheet number: ' + match.group(1))
                if int(match.group(1)) > int(stop_at):
                    continue

            #print(s)
            sheet_name = s

            #print(name_cols)
            name_df = xl.parse(sheet_name, usecols = name_cols,header=1)
            #display(name_df.head())

            # xl.parse takes 1st ro parsed into column headers - so we retrieve company name
            #  from the name of the column
            company_name = name_df.columns[2]
            #print(company_name)

            # xl.parse takes 1st ro parsed into column headers - so we retrieve regsitry number
            #  from the row where "Company name:" column is equal to "Registry number"
            if 'Company name:' in name_df.columns:
                company_number = name_df[name_df['Company name:'] == 'Registry number'][company_name]
            else:
                company_number = name_df[name_df['Company name'] == 'Registry number'][company_name]

            #print(company_name, company_number)

            df = xl.parse(sheet_name, skiprows=data_skip_rows, parse_cols = data_cols, header=None)        
            
            
            for i in df.iloc[0:1]:
                if df.iloc[0:1][i][0] == 'Company':
                    df.iloc[0:1][i][0] = 'per company'
                if df.iloc[0:1][i][0] == 'Government Agency':
                    df.iloc[0:1][i][0] = 'per government'
                    
            
            # fill out cells with merged column headers
            df.iloc[0:2] = df.iloc[0:2].fillna(method='ffill', axis=1)
            df.columns = df.iloc[0:2].apply(join_column_titles, axis=0)
            df = df.iloc[2:]
            df = df.reset_index(drop=True)
            df.columns = rename_duplicate_column_titles(df.columns)
            df.columns = [x.lower() for x in df.columns]
            df.rename(columns={'final difference.final': 'final difference', \
                               'comment.final': 'comment'}, inplace=True)
            if 'n.n' in df.columns:
                df.rename(columns={'n.n': 'n'}, inplace=True) 
            if 'description of payment.description' in df.columns:
                df.rename(columns={'description of payment.description': \
                                   'description of payment'}, inplace=True)
                
            for col in df.columns:
                new_col_name = col.replace("governement", "government")
                new_col_name = new_col_name.replace("ajust", "adjust")
                df.rename(columns={col: new_col_name}, inplace=True)
            
            if 'per company.initial' in df.columns:
                df.rename(columns={'per company.initial': \
                                   'per company.original'}, inplace=True)
            if 'per government.initial' in df.columns:
                df.rename(columns={'per government.initial': \
                                   'per government.original'}, inplace=True)
                
                
            if 'company.initial' in df.columns:
                df.rename(columns={'company.initial': \
                                   'per company.original'}, inplace=True)
            if 'government agency.initial' in df.columns:
                df.rename(columns={'government agency.initial': \
                                   'per government.original'}, inplace=True)


            #display(df.head())
            # clean name for the company to save individual CSV file if needed
            name = map(lambda x: ''.join(e for e in x if e.isalnum()) , company_name.split(' '))
            name = ' '.join(w for w in name)
            name = re.sub( '\s+', ' ', name.strip())
            filename = re.sub( '\s+', '_', name.strip())+'.csv'
            #         df.to_csv(filename)

            #print(filename)

            df_all = add_sheet_to_main_df(df_all,df, company_name,key_terms)
    
        df_all = df_all.reset_index(drop=True)
        df_all.to_csv(year+'/'+output_file+'.csv', encoding='utf-8')

        display(df_all.head())


In [48]:
read_files(files_to_read,df_all)

2014-2015/Annex 18 - Reconciliation sheets Gems & Jade 14-15.xlsx
['C (1)', 'C (2)', 'C (3)', 'C (4)', 'C (5)', 'C (6)', 'C (7)', 'C (8)', 'C (9)', 'C (10)', 'C (11)', 'C (12)', 'C (13)', 'C (14)', 'C (15)', 'C (16)', 'C (17)', 'C (18)', 'C (19)', 'C (20)', 'C (21)', 'C (22)', 'C (23)', 'C (24)', 'C (25)', 'C (26)', 'C (27)', 'C (28)', 'C (29)', 'C (30)', 'C (31)', 'C (32)', 'C (33)', 'C (34)', 'C (35)', 'C (36)', 'C (37)', 'C (38)', 'C (39)', 'C (40)', 'C (41)', 'C (42)', 'C (43)', 'C (44)', 'C (45)', 'C (46)', 'C (47)', 'C (48)', 'C (49)', 'C (50)', 'C (51)', 'C (52)', 'C (53)', 'C (54)', 'C (55)', 'C (56)', 'C (57)', 'C (58)', 'C (59)', 'C (60)', 'C (61)', 'C (62)', 'C (63)', 'C (64)', 'C (65)', 'C (66)', 'C (67)', 'C (68)', 'C (69)', 'C (70)', 'C (71)', 'C (72)']
current sheet number: 1
current sheet number: 2
current sheet number: 3
current sheet number: 4
current sheet number: 5
current sheet number: 6
current sheet number: 7
current sheet number: 8
current sheet number: 9
curren

,company_name,name_of_revenue_stream,paid_to,payment_category,units,per_company_original,per_company_adjust,per_company_final,per_government_original,per_government_adjust,per_government_final,final_difference,comment
0,Myanmar Win Gate Gems & Jewellery Mining,Production Split (Government and SOEs share),Ministry of Natural Resources and Environmenta...,payments in kind,MMK,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,
1,Myanmar Win Gate Gems & Jewellery Mining,Royalties on production - 20% collected by/pai...,Ministry of Natural Resources and Environmenta...,payments in cash,MMK,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,
2,Myanmar Win Gate Gems & Jewellery Mining,Royalties on production- 20% collected by/paid...,Ministry of Natural Resources and Environmenta...,payments in cash,MMK,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,
3,Myanmar Win Gate Gems & Jewellery Mining,Sale Split - 40%,Ministry of Natural Resources and Environmenta...,payments in cash,MMK,2.201612e+10,0.0,2.201612e+10,2.201612e+10,0.0,2.201612e+10,0.0,
4,Myanmar Win Gate Gems & Jewellery Mining,Royalties on Sales- 7%,Ministry of Natural Resources and Environmenta...,payments in cash,MMK,4.368923e+09,0.0,4.368923e+09,4.368923e+09,0.0,4.368923e+09,0.0,


2014-2015/Annex 18 - Reconciliation sheets Oil & Gas 14-15.xlsx
['RT (1)', 'RT (2)', 'RT (3)', 'RT (4)', 'RT (5)', 'RT (6)', 'RT (7)', 'RT (8)', 'RT (9)', 'RT (10)', 'RT (11)', 'RT (12)', 'RT (13)', 'RT (14)', 'RT (15)', 'RT (16)', 'RT (17)', 'RT (18)', 'RT (19)', 'RT (20)', 'RT (21)', 'RT (22)', 'RT (23)', 'RT (24)', 'RT (25)', 'RT (26)', 'RT (27)', 'RT (28)', 'RT (29)', 'RT (30)', 'RT (31)', 'RT (32)', 'RT (33)', 'RT (34)', 'RT (35)']
current sheet number: 1
current sheet number: 2
current sheet number: 3
current sheet number: 4
current sheet number: 5
current sheet number: 6
current sheet number: 7
current sheet number: 8
current sheet number: 9
current sheet number: 10
current sheet number: 11
current sheet number: 12
current sheet number: 13
current sheet number: 14
current sheet number: 15
current sheet number: 16
current sheet number: 17
current sheet number: 18
current sheet number: 19
current sheet number: 20
current sheet number: 21
current sheet number: 22
current sheet numb

,company_name,name_of_revenue_stream,paid_to,payment_category,units,per_company_original,per_company_adjust,per_company_final,per_government_original,per_government_adjust,per_government_final,final_difference,comment
0,Petronas Carigali Hong Kong Ltd (PCML),State production entitelment,,payments in kind,in barils,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,Petronas Carigali Hong Kong Ltd (PCML),MOGE production entitelment,,payments in kind,in barils,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,Petronas Carigali Hong Kong Ltd (PCML),Royalties,,payments in kind,in barils,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,Petronas Carigali Hong Kong Ltd (PCML),Baseline payment (IPR/PCC),,payments in kind,in barils,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,Petronas Carigali Hong Kong Ltd (PCML),Risk compensation (IPR/PCC),,payments in kind,in barils,0.0,0.0,0.0,0.0,0.0,0.0,0.0,


2014-2015/Annex 18 - Reconciliation sheets Other minerals 14-15.xlsx
['Companies List', 'suivi Hard copy', 'Unrec diff by Comp (2)', 'unrec database', 'RT (1)', 'RT (2)', 'RT (3)', 'RT (4)', 'RT (5)', 'RT (6)', 'RT (7)', 'RT (8)', 'RT (9)', 'RT (10)', 'RT (11)', 'RT (12)', 'RT (13)', 'RT (14)', 'RT (15)', 'RT (16)', 'RT (17)', 'RT (18)', 'RT (19)', 'RT (20)', 'RT (21)', 'RT (22)', 'RT (23)', 'RT (24)', 'RT (25)', 'RT (26)', 'RT (27)', 'RT (28)', 'RT (29)', 'License', 'Ownership(old)', 'Report database', 'Uni disc']
current sheet number: 1


...

,company_name,name_of_revenue_stream,paid_to,payment_category,units,per_company_original,per_company_adjust,per_company_final,per_government_original,per_government_adjust,per_government_final,final_difference,comment
0,Petronas Carigali Hong Kong Ltd (PCML),State production entitelment,,payments in kind,in barils,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,Petronas Carigali Hong Kong Ltd (PCML),MOGE production entitelment,,payments in kind,in barils,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,Petronas Carigali Hong Kong Ltd (PCML),Royalties,,payments in kind,in barils,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,Petronas Carigali Hong Kong Ltd (PCML),Baseline payment (IPR/PCC),,payments in kind,in barils,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,Petronas Carigali Hong Kong Ltd (PCML),Risk compensation (IPR/PCC),,payments in kind,in barils,0.0,0.0,0.0,0.0,0.0,0.0,0.0,


2015-2016/Annex 18 - Reconciliation sheets Oil & Gas transp 15-16.xlsx
['RT (1)', 'RT (2)', 'RT (3)', 'RT (4)', 'RT (5)']
current sheet number: 1
current sheet number: 2
current sheet number: 3
current sheet number: 4
current sheet number: 5


,company_name,name_of_revenue_stream,paid_to,payment_category,units,per_company_original,per_company_adjust,per_company_final,per_government_original,per_government_adjust,per_government_final,final_difference,comment
0,Andaman Transportation Limited (ATL),State production entitelment,,payments in kind,in barils,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,Andaman Transportation Limited (ATL),MOGE production entitelment,,payments in kind,in barils,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,Andaman Transportation Limited (ATL),Royalties,,payments in kind,in barils,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,Andaman Transportation Limited (ATL),Baseline payment (IPR/PCC),,payments in kind,in barils,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,Andaman Transportation Limited (ATL),Risk compensation (IPR/PCC),,payments in kind,in barils,0.0,0.0,0.0,0.0,0.0,0.0,0.0,


2015-2016/Annex 18 - Reconciliation sheets Other minerals 15-16.xlsx
['Companies List', 'RT (1)', 'RT (2)', 'RT (3)', 'RT (4)', 'RT (5)', 'RT (6)', 'RT (7)', 'RT (8)', 'RT (9)', 'RT (10)', 'RT (11)', 'RT (12)', 'RT (13)', 'RT (14)', 'RT (15)', 'RT (16)', 'RT (17)', 'RT (18)', 'RT (19)', 'RT (20)', 'RT (21)', 'RT (22)', 'RT (23)', 'RT (24)', 'RT (25)', 'RT (26)', 'RT (27)', 'RT (28)', 'RT (29)', 'License', 'Ownership(old)', 'Uni disc', 'Adjt database', 'Total Adj', 'Adj per tax (C)', 'Adj per Comp (C)', 'Adj per tax (Gov)', 'Adj per Comp (Gov)']
current sheet number: 1
current sheet number: 2
current sheet number: 3
current sheet number: 4
current sheet number: 5
current sheet number: 6
current sheet number: 7
current sheet number: 8
current sheet number: 9
current sheet number: 10
current sheet number: 11
current sheet number: 12
current sheet number: 13
current sheet number: 14
current sheet number: 15
current sheet number: 16
current sheet number: 17
current sheet number: 18
current 

,company_name,name_of_revenue_stream,paid_to,payment_category,units,per_company_original,per_company_adjust,per_company_final,per_government_original,per_government_adjust,per_government_final,final_difference,comment
0,San Na DiCo. Ltd.,Production Split (Government and SOEs share),,payments in kind,gold in t.oz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,San Na DiCo. Ltd.,Royalties,,payments in kind,gold in t.oz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,San Na DiCo. Ltd.,Corporate Income Tax (CIT),MoF-IRD-Customs Department,payments in cash,MMK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,San Na DiCo. Ltd.,Commercial Tax,MoF-IRD-Customs Department,payments in cash,MMK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,San Na DiCo. Ltd.,Commercial Tax on Imports,MoF-IRD-Customs Department,payments in cash,MMK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
